In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from pathlib import Path
from natsort import natsort_keygen, natsorted # to naturally sort string columns

plt.style.use('matplotlibrc')

KB = 1024
MB = 1024 * 1024
GB = 1024 * 1024 * 1024

In [ ]:
def set_size(fraction_width=0.95, fraction_height=0.25):
    width_pt = 241.14749 # column width in pt
    height_pt = 626.0 # page height in pt

    fig_width_pt = width_pt * fraction_width
    fig_height_pt = height_pt * fraction_height
    inches_per_pt = 1 / 72.27

    fig_width_in = fig_width_pt * inches_per_pt
    fig_height_in = fig_height_pt * inches_per_pt

    return (fig_width_in, fig_height_in)

In [ ]:
# define color to use throughout the notebook
paired = matplotlib.colormaps['Paired']
tab20 = matplotlib.colormaps['tab20']
dark2 = matplotlib.colormaps['Dark2']
tab20c = matplotlib.colormaps['tab20c']

c_st =  tab20c(0)
c_st_tt = tab20c(1)
c_decomp = tab20c(3)

c_rdb = tab20c(5)
c_rdb_tt = tab20c(6)
c_rdb_pj = tab20c(7)

c_rdb_w_pj = tab20(2)
c_rdb_wo_pj = tab20(4)
c_redundancy = tab20(14)

c_rm1 = dark2(0)
c_rm2 = dark2(2)
c_rm3 = dark2(5)
c_rm4 = dark2(3)

colormap = {
    'Single Table': c_st,
    'ST transfer time': c_st_tt,
    'Decompose': c_decomp,
    'RDB w/ post-join info': c_rdb_w_pj,
    'Result DB': c_rdb,
    'RDB transfer time': c_rdb_tt,
    'RDB post-join': c_rdb_pj,
    'RDB w/o post-join info': c_rdb_wo_pj,
    '0. Single Table': c_st,
    'RM1. Dynamic SELECT DISTINCT': c_rm1,
    'RM2. Materialized SELECT DISTINCT': c_rm2,
    'RM3. Dynamic Subquery': c_rm3,
    'RM4. Materialized Subquery': c_rm4,
    'Redundancy': c_redundancy
}

In [ ]:
# JOB queries
job_postgres_queries = [
    "q1b",
    "q2a",
    "q3c",
    "q4a",
    "q5c",
    "q6a",
    "q7a",
    "q8a",
    "q9c",
    "q10c",
    "q11c",
    "q12a",
    "q13b",
    "q14a",
    "q15d",
    "q16b",
    "q17a",
    "q18c",
    "q19a",
    "q20b",
    "q21a",
    "q22c",
    "q23a",
    "q24a",
    "q25b",
    "q26a",
    "q27a",
    "q28c",
    "q29a",
    "q30c",
    "q31a",
    "q32a",
    "q33c",
]

job_mutable_queries = [
    "q1b",
    "q2a",
    "q3c",
    "q4a",
    "q5c",
    "q7a",
    "q8a",
    "q9c",
    "q10c",
    "q11c",
    "q12a",
    "q14a",
    "q15d",
    "q18c",
    "q19a",
    "q21a",
    "q22c",
    "q23a",
    "q24a",
    "q25b",
    "q26a",
    "q27a",
    "q28c",
    "q30c",
    "q31a",
    "q33c",
]

job_subset = [
    "q3c",
    "q4a",
    "q9c",
    "q11c",
    "q16b",
    "q18c",
    "q22c",
    "q25b",
    "q28c",
    "q33c",
]

## Join-Order Benchmark

In [ ]:
def compression_ratio(uncompressed, compressed):
    """
    Compute the compression ratio.

    Returns:
    float: Compression ratio.
    """
    # assert uncompressed != 0, "uncompressed must not be zero"
    if compressed == 0:
        return 0
    return round(uncompressed / compressed, 2)

def to_latex(single_table, rdb_w_post_join_info, rdb_wo_post_join_info, entries_per_row):
    table_top = f"\\begin{{table*}}\n"
    table_top += f"\t\\small\n"
    table_top += f"\t\\caption{{JOB result set sizes in KiB (compression ratio).}}\n"
    table_top += f"\t\\label{{tab:job-result-set-sizes}}\n"

    tabular = ""
    queries = natsorted(single_table['query'].unique())
    assert(natsorted(single_table['query'].unique()) == natsorted(rdb_w_post_join_info['query'].unique()) == natsorted(rdb_wo_post_join_info['query'].unique()))
    num_queries = len(queries)
    assert num_queries % entries_per_row == 0, "number of queries should be a multiple of entries per row"
    num_tabulars = num_queries // entries_per_row
    for t in range(num_tabulars):
        tabular_queries = queries[t * entries_per_row:t * entries_per_row + entries_per_row]
        tabular += f"\t\\begin{{tabular}}{{\n"
        tabular += f"\t\t|l!{{\\vrule width 1.5pt}}\n"
        for e in range(entries_per_row):
            tabular += f"\t\tR{{\\cellwidth}}R{{\\cellwidthratio}}|\n"
        tabular += f"\t\t}}\n"
        tabular += f"\t\t\\hline\n"
        tabular += f"\t\t\\multicolumn{{1}}{{|c!{{\\vrule width 1.5pt}}}}{{\\textbf{{Method}}}}\n"
        for q in tabular_queries:
            tabular += f"\t\t& \\multicolumn{{2}}{{c|}}{{\\textbf{{{q[1:]}}}}}\n"
        tabular += f"\t\t\\\\ \\noalign{{\\hrule height 1.5pt}}\n"
        single_table_data = "\t\tST"
        rdb_rp = "\t\tRDB$_{\\text{RP}}$"
        rdb = "\t\tRDB"
        for q in tabular_queries:
            single_table_data  += f" & {round(single_table[single_table['query'] == q]['size'].values[0], 2)} & \\hspace{{\\gap}}(1.0)"
            rdb_rp += f" & {round(rdb_w_post_join_info[rdb_w_post_join_info['query'] == q]['size'].values[0], 2)} & \\hspace{{\\gap}}({round(rdb_w_post_join_info[rdb_w_post_join_info['query'] == q]['compression_ratio'].values[0], 1)})"
            rdb += f" & {round(rdb_wo_post_join_info[rdb_wo_post_join_info['query'] == q]['size'].values[0], 2)} & \\hspace{{\\gap}}({round(rdb_wo_post_join_info[rdb_wo_post_join_info['query'] == q]['compression_ratio'].values[0], 1)})"
        tabular += f"{single_table_data}\\\\\\hline\n"
        tabular += f"{rdb_rp}\\\\\\hline\n"
        tabular += f"{rdb}\\\\\\hline\n"
        tabular += f"\t\\end{{tabular}}\n"
    table_bottom = f"\\end{{table*}}"
    latex = f"{table_top}{tabular}{table_bottom}"
    print(latex)

data = pd.read_csv('../result-set-sizes/job/result-set-sizes.csv')
data = data.drop(['relation', 'count'], axis=1)
data = data.groupby(by=['database', 'query', 'method'], as_index=False).sum()
data = data.sort_values(by=['query', 'method'], key=natsort_keygen())
data['size'] = round(data['size'] / KB, 2)

queries = job_subset

data = data[data['query'].isin(queries)]

single_table = data[data['method'] == 'Single Table']
rdb_w_post_join_info = data[data['method'] == 'rdb_w_post_join_info']
rdb_w_post_join_info['compression_ratio'] = [ compression_ratio(baseline, new) for baseline, new in zip(single_table['size'], rdb_w_post_join_info['size']) ]
rdb_wo_post_join_info = data[data['method'] == 'rdb_wo_post_join_info']
rdb_wo_post_join_info['compression_ratio'] = [ compression_ratio(baseline, new) for baseline, new in zip(single_table['size'], rdb_wo_post_join_info['size']) ]
 
# display(single_table)
# display(rdb_wo_post_join_info)
# display(rdb_wo_post_join_info)
to_latex(single_table, rdb_w_post_join_info, rdb_wo_post_join_info, 5)

## Synthetic Star Schema (S3)

In [ ]:
def compute_s3_result_set_sizes():
    num_dim_tables = 4 # number of dimension tables
    dim_payload = 20 # byte
    dim_row_size = 4 + dim_payload # 4 byte primary key + payload
    dim_table_size = 60
    selectivities = np.arange(0.1, 1.1, 0.1)
    dim_table_sizes = [ int(dim_table_size * sel) for sel in selectivities ]

    fact_payload = 20
    fact_row_size = 4 + num_dim_tables * 4 + fact_payload # 4 byte pk + 4 byte foreign key per dimension table + payload
    fact_size = dim_table_size**num_dim_tables
    fact_table_sizes = [ dim_size**num_dim_tables for dim_size in dim_table_sizes ]

    single_table = [ ((fact_row_size + dim_row_size * num_dim_tables) * f_size) / MB for f_size in fact_table_sizes ]
    rdb_with_post_join = ([ ((f_size * fact_row_size) + (dim_row_size * d_size * num_dim_tables)) / MB
                           for f_size, d_size in zip(fact_table_sizes, dim_table_sizes) ])
    rdb_without_post_join = [ (dim_payload * d_size * num_dim_tables + f_size * (fact_payload)) / MB for f_size, d_size in zip(fact_table_sizes, dim_table_sizes) ]

    return single_table, rdb_with_post_join, rdb_without_post_join

single_table, rdb_with_post_join, rdb_without_post_join = compute_s3_result_set_sizes()
selectivities = np.arange(0.1, 1.1, 0.1)

### Plot
fig, ax = plt.subplots(figsize=set_size(fraction_width=1.0, fraction_height=0.2), layout='constrained')
ax.plot(selectivities, single_table, 'o--', color=colormap['Single Table'], label='Single Table')
ax.plot(selectivities, rdb_with_post_join, 'x--', color=colormap['RDB w/ post-join info'], label=r'RDB$_{\text{RP}}$')
ax.plot(selectivities, rdb_without_post_join, '^--', color=colormap['RDB w/o post-join info'], label='RDB')

ax.fill_between(selectivities, single_table, rdb_with_post_join, color=colormap['Redundancy'], label='Denormalization redundancy')

ax.set_xlabel('Filter selectivity on dimension tables')
ax.set_ylabel('Result set size [MiB]')
ax.tick_params(axis='y')
ax.legend(loc='upper left', ncols=1)

fig.savefig('synthetic-result-sizes.pdf', bbox_inches='tight')

# Rewrite Methods

## Join-Order Benchmark

In [ ]:
def preprocess_data(data):
    # construct the following dictionary:
    # {
    #   'RM0': [q0_time, q1_time, ..., qn-1_time]
    #   'RM1': [q0_time, q1_time, ..., qn-1_time]
    # }
    data = data.sort_values(by=['query', 'method'], key=natsort_keygen()) # ensure that data is sorted correctly
    method_times = defaultdict(list)
    for _, row in data.iterrows():
        method_times[row['method']].append(row['time'])
    return method_times

def bar_plot(data, data_transfer, filename):
    queries = natsorted(data['query'].unique())
    queries = [ q[1:] for q in queries ]
    method_times = preprocess_data(data[data['data_transfer'] == data_transfer])
    x = np.arange(len(queries))  # the label locations
    width = 0.18  # the width of the bars
    multiplier = 0
    fig, ax = plt.subplots(figsize=set_size(fraction_width=2, fraction_height=0.21), layout='constrained')
    for method, times in method_times.items():
        offset = width * multiplier
        if method == 'RM1. Dynamic SELECT DISTINCT':
            rects = ax.bar(x + offset, times, width, color=colormap[method], label=r'RM1. Dynamic \texttt{SELECT DISTINCT}')      
        elif method == 'RM2. Materialized SELECT DISTINCT':
            rects = ax.bar(x + offset, times, width, color=colormap[method], label=r'RM2. Materialized \texttt{SELECT DISTINCT}')
        else:
            rects = ax.bar(x + offset, times, width, color=colormap[method], label=method)
        multiplier += 1
    ax.set_xticks(x + width + width/2, queries)
    ax.set_xlabel('JOB queries')
    ax.legend(loc='upper left')
    ax.xaxis.grid(False) # Disable grid lines on the x-axis

    log_scale = True
    if log_scale:
        ax.set_ylabel('Query execution time [ms]')
        ax.set_yscale('log')
    else:
        ax.set_ylabel('Query execution time [ms]')
        ax.set_ylim(0)

    fig.savefig(filename, bbox_inches='tight')

data = pd.read_csv('../rewrite-methods/job/rewrite-results.csv')
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer', 'run'], as_index=False).sum()
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer'], as_index=False).median().drop(['run', 'num_query_internal'], axis=1)
data = data.drop(['database', 'system'], axis=1)
include_queries = job_postgres_queries
data = data[data['query'].isin(include_queries)]
exclude_methods = [
    '0. Single Table'
]
data = data[~data['method'].isin(exclude_methods)]

bar_plot(data, False, 'job-rewrite-methods.pdf')

In [ ]:
def overhead(baseline, new):
    assert baseline != 0, "baseline must not be zero"
    if (new <= baseline): # new is faster -> improvement
        return -((baseline - new) / baseline) * 100
    else: # new is slower -> overhead 
        return ((new - baseline) / baseline) * 100

def to_latex(single_table, best_rewrite_method, entries_per_row):
    table_top = f"\\begin{{table*}}\n"
    table_top += f"\t\\small\n"
    table_top += f"\t\\caption{{Overhead of the \\textbf{{best}} rewrite method compared to the single-table execution time for the IMDb dataset.}} \n"
    table_top += f"\t\\label{{tab:job-overhead-rewrites}}\n"
    
    tabular = ""
    queries = natsorted(single_table['query'].unique())
    assert natsorted(single_table['query'].unique()) == natsorted(best_rewrite_method['query'].unique())
    num_queries = len(queries)
    assert num_queries % entries_per_row == 0, "number of queries should be a multiple of entries per row"
    num_tabulars = num_queries // entries_per_row
    for t in range(num_tabulars):
        tabular_queries = queries[t * entries_per_row:t * entries_per_row + entries_per_row]
        tabular += f"\t\\begin{{tabular}}{{\n"
        tabular += f"\t\t|l!{{\\vrule width 1.5pt}}\n"
        for e in range(entries_per_row):
            tabular += f"\t\tP{{\\cellwidthperfratio}}|\n"
        tabular += f"\t\t}}\n"
        tabular += f"\t\t\\hline\n"
        tabular += f"\t\t\\textbf{{JOB Query}}\n"
        for q in tabular_queries:
            tabular += f"\t\t& \\textbf{{{q[1:]}}}\n"
        tabular += f"\t\t\\\\ \\noalign{{\\hrule height 1.5pt}}\n"
        overhead_data = "\t\tOverhead"
        best_rm_data = "\t\tBest RM"
        for q in tabular_queries:
            overhead_value = round(overhead(int(single_table[single_table['query'] == q]['time'].values[0]), int(best_rewrite_method[best_rewrite_method['query'] == q]['time'].values[0])), 1)
            overhead_data  += f" & {overhead_value}\\%"
            best_rm = best_rewrite_method[best_rewrite_method['query'] == q]['method'].values[0]
            best_rm_data += f" & RM~{best_rm[2]}"
        tabular += f"{overhead_data}\\\\\n"
        tabular += f"{best_rm_data}\\\\\\hline\n"
        tabular += f"\t\\end{{tabular}}\n"
    table_bottom = f"\\end{{table*}}\n"
    latex = f"{table_top}{tabular}{table_bottom}"
    print(latex)

data = pd.read_csv('../rewrite-methods/job/rewrite-results.csv')
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer', 'run'], as_index=False).sum()
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer'], as_index=False).median().drop(['run', 'num_query_internal'], axis=1)
data = data.drop(['database', 'system'], axis=1)
data = data[data['data_transfer'] == False]
include_queries = job_postgres_queries
data = data[data['query'].isin(include_queries)]

single_table = data[data['method'] == '0. Single Table']
rewrite_methods = data[data['method'] != '0. Single Table']

# NOTE: be careful as ties are included (handle them manually)
min_values = rewrite_methods.groupby(by=['query'])['time'].transform('min')
best_rewrite_method = rewrite_methods[rewrite_methods['time'] == min_values]

# ensure that both dataframes are sorted correctly
single_table = single_table.sort_values(by=['query', 'method'], key=natsort_keygen()) # ensure that data is sorted correctly
best_rewrite_method = best_rewrite_method.sort_values(by=['query', 'method'], key=natsort_keygen()) # ensure that data is sorted correctly

assert len(single_table) == len(best_rewrite_method), f"{len(single_table)} vs {len(best_rewrite_method)}"

to_latex(single_table, best_rewrite_method, 11)

# RESULTDB Algorithm

## Join-Order Benchmark

In [ ]:
# construct the following dictionary:
# {
#   'Single-Table': [q4_time, q5_time, ...]
#   'Single-Table + Decompose': [q4_time, q5_time, ...]
#   'Result-DB': [q4_time, q5_time, ...]
# }
# ensure that `queries` is sorted!
queries = job_mutable_queries

single_table_decompose = []
resultdb = []
processed_queries = []
num_queries = 0
for query in queries:
    result_file = Path(f"../algorithm/job/{query}_results.csv")
    if not result_file.is_file():
        print(f"{result_file} does not exist. Skipping...")
        continue
    processed_queries.append(query)
    data = pd.read_csv(result_file)  
    data = (data.groupby(by=['commit', 'date', 'version', 'suite', 'benchmark', 'experiment', 'name', 'config', 'case',], as_index=False)
                .median()
                .drop(['commit', 'date', 'version', 'suite', 'benchmark', 'case', 'runid'], axis=1)
           )
    experiment = data['experiment'].unique()
    assert len(experiment) == 1, f'experiment contains multiple different queries'
    assert experiment[0] == query, f'experiment query: {experiment[0]} does not match query {query} of current file'

    for _, row in data.iterrows():
        algorithm = row['name']
        execution_time = row['time']
        if "single-table" in algorithm:
            single_table_execution_time = execution_time
        elif "decompose" in algorithm:
            decompose_execution_time = execution_time
        elif "resultdb" in algorithm:
            resultdb_execution_time = execution_time
        else:
            assert False, "experiment name: {name} does not match any of our algorithms"

    single_table_decompose.append((single_table_execution_time, decompose_execution_time - single_table_execution_time))
    resultdb.append(resultdb_execution_time)

assert len(single_table_decompose) == len(resultdb) == len(processed_queries), f'number of measurements has to match the processed queries'

# plot data
x = np.arange(len(processed_queries))  # the label locations
width = 0.30  # the width of the bars
fig, main_ax = plt.subplots(figsize=set_size(fraction_width=1, fraction_height=0.21), layout='constrained')

st_time = [ st_decomp[0] for st_decomp in single_table_decompose ]
decomp_time = [ st_decomp[1] for st_decomp in single_table_decompose ]
# ZOOM IN
inset_ax = main_ax.inset_axes([0.4, 0.35, 0.1, 0.2],
                              xlim=[12.8, 13.5], ylim=[1520, 1540],
                              xticks = [],
                              xticklabels=[],
                              )
for ax in main_ax, inset_ax:
    ax.bar(x, st_time, width, color=colormap['Single Table'], edgecolor='black', label='Single Table')
    ax.bar(x, decomp_time, width, bottom=st_time, color=colormap['Decompose'], edgecolor='black', label='Decompose')
    offset = width
    ax.bar(x + offset, resultdb, width, color=colormap['Result DB'], edgecolor='black', label=r'\textsc{ResultDB$_{\text{semi-join}}$}')

main_ax.indicate_inset_zoom(inset_ax)

main_ax.set_ylabel('Query execution time [ms]')
main_ax.set_xlabel('JOB queries')
processed_queries = [ q[1:] for q in processed_queries ]
main_ax.set_xticks(x + width / 2, processed_queries, rotation=90)
main_ax.legend(loc='upper left')
main_ax.set_ylim(0)
main_ax.xaxis.grid(False) # Disable grid lines on the x-axis

fig.savefig('job-algorithm.pdf', bbox_inches='tight')

# Post-join


## Join-Order Benchmark

In [ ]:
queries = [q for q in job_subset if q != "q16b"]

post_join_times = []
processed_queries = []
num_queries = 0
for query in queries:
    result_file = Path(f"../post-join/job/{query}/{query}_results.csv")
    if not result_file.is_file():
        print(f"{result_file} does not exist. Skipping...")
        continue
    processed_queries.append(query)
    data = pd.read_csv(result_file)  
    data = (data.groupby(by=['commit', 'date', 'version', 'suite', 'benchmark', 'experiment', 'name', 'config', 'case',], as_index=False)
                .median()
                .drop(['commit', 'date', 'version', 'suite', 'benchmark', 'case', 'runid'], axis=1)
           )
    experiment = data['experiment'].unique()
    assert len(experiment) == 1, f'experiment contains multiple different queries'
    assert experiment[0] == query, f'experiment query: {experiment[0]} does not match query {query} of current file'

    assert data.shape[0] == 1, f'after aggregating, dataframe should only contain one row'
    
    post_join_times.append(data.iloc[0]['time'])
    
# plot data
x = np.arange(len(processed_queries))  # the label locations
width = 0.30  # the width of the bars
fig, ax = plt.subplots(figsize=set_size(fraction_width=1, fraction_height=0.20), layout='constrained')

ax.bar(x, post_join_times, color=colormap['Single Table'], label='Single Table')

ax.set_ylabel('Query execution time [ms]')
ax.set_title('Post-join in mutable')
ax.set_xlabel('JOB queries')
ax.set_xticks(x, queries)
ax.legend(loc='upper right')
ax.set_ylim(0)
ax.xaxis.grid(False) # Disable grid lines on the x-axis

fig.savefig('job-postjoin-mutable.pdf', bbox_inches='tight')

In [ ]:
data = pd.read_csv('../post-join/job/postjoin-postgres-results.csv')
data = data.groupby(by=['database', 'system', 'method', 'query'], as_index=False).median().drop(['database', 'system', 'method', 'run'], axis=1)
data = data.sort_values(by=['query'], key=natsort_keygen())

queries = data['query']
x = np.arange(len(queries))
postjoin_times = data['time']

fig, ax = plt.subplots(figsize=set_size(fraction_width=1, fraction_height=0.20), layout='constrained')
ax.bar(x, postjoin_times, color=colormap['Single Table'], label='Single Table')

ax.set_ylabel('Query execution time [ms]')
ax.set_title('Post-join in PostgreSQL')
ax.set_xlabel('JOB queries')
ax.set_xticks(x, queries)
ax.legend(loc='upper right')
ax.set_ylim(0)
ax.xaxis.grid(False) # Disable grid lines on the x-axis

fig.savefig('job-postjoin-postgres.pdf', bbox_inches='tight')

# End-to-End Runtime Including Query Execution, Data Transfer, & Post-join

## Rewrite Methods

### Join-Order Benchmark

In [ ]:
def bar_plot(data, filename):
    queries = data['query']
    x = np.arange(len(queries))  # the label locations
    width = 0.20  # the width of the bars
    fig, ax = plt.subplots(figsize=set_size(fraction_width=1, fraction_height=0.2), layout='constrained')

    # Single Table
    ax.bar(x, data['st_exec_time'], width, color=colormap['Single Table'], edgecolor='black',label='Single Table')
    ax.bar(x, data['st_transfer_time'],  width=width, bottom=data['st_exec_time'], color=colormap['ST transfer time'], edgecolor='black', label='Single Table Transfer Time')

    # Rewrite Method
    ax.bar(x + width, data['rm_exec_time'], width=width, color=colormap['Result DB'], edgecolor='black', label='Result DB')
    ax.bar(x + width, data['rm_transfer_time'], width=width, bottom=data['rm_exec_time'], color=colormap['RDB transfer time'], edgecolor='black', label='Result DB Transfer Time')
    ax.bar(x + width, data['rm_post_join_time'], width=width, bottom=data['rm_exec_time'] + data['rm_transfer_time'], color=colormap['RDB post-join'], edgecolor='black', label='Result DB Post-join')

    ax.set_xticks(x + width / 2, queries)
    ax.set_xlabel('JOB queries')
    ax.legend(loc='upper left')
    ax.xaxis.grid(False)  # Disable grid lines on the x-axis

    log_scale = False
    if log_scale:
        ax.set_ylabel('Runtime [ms] (log-scale)')
        ax.set_yscale('log')
    else:
        ax.set_ylabel('Runtime [ms]')
        ax.set_ylim(0)

    fig.savefig(filename, bbox_inches='tight')

def to_latex(data, entries_per_row):
    table_top = f"\\begin{{table*}}\n"
    table_top += f"\t\\small\n"
    table_top += f"\t\\caption{{End-to-end performance of the best rewrite method~(RM) compared to the Single Table (ST) execution on JOB.}} \n"
    table_top += f"\t\\label{{tab:job-end-to-end-rewrite-methods}}\n"

    tabular = ""
    queries = natsorted(data['query'].unique())
    num_queries = len(queries)
    assert num_queries % entries_per_row == 0, "number of queries should be a multiple of entries per row"
    num_tabulars = num_queries // entries_per_row
    for t in range(num_tabulars):
        tabular_queries = queries[t * entries_per_row:t * entries_per_row + entries_per_row]
        tabular += f"\t\\begin{{tabular}}{{\n"
        tabular += f"\t\t|l!{{\\vrule width 1.5pt}}\n"
        for e in range(entries_per_row):
            if e != entries_per_row - 1:
                tabular += f"\t\tR{{\\cellwidth}}|R{{\\cellwidth}}!{{\\vrule width \\vw}}\n"
            else:
                tabular += f"\t\tR{{\\cellwidth}}|R{{\\cellwidth}}|}}\n"
        tabular += f"\t\t\\hline\n"
        tabular += f"\t\t\\textbf{{JOB Query}}\n"
        for i, q in enumerate(tabular_queries):
            if i != entries_per_row - 1:
                tabular += f"\t\t& \\multicolumn{{2}}{{c!{{\\vrule width \\vw}}}}{{\\textbf{{{q[1:]}}}}}\n"
            else:
                tabular += f"\t\t& \\multicolumn{{2}}{{c|}}{{\\textbf{{{q[1:]}}}}}\n"
        tabular += f"\t\t\\\\\\hline\n"
        tabular += f"\t\t\\textbf{{Approach}}"
        for e in range(entries_per_row):
            if e != entries_per_row - 1:
                tabular += f"\t\t& \\multicolumn{{1}}{{c|}}{{\\textbf{{ST}}}} & \\multicolumn{{1}}{{c!{{\\vrule width \\vw}}}}{{\\textbf{{RM}}}}\n"
            else:
                tabular += f"\t\t& \\multicolumn{{1}}{{c|}}{{\\textbf{{ST}}}} & \\multicolumn{{1}}{{c|}}{{\\textbf{{RM}}}}\n"
        tabular += f"\t\t\\\\ \\noalign{{\\hrule height 1.5pt}}\n"
        query_execution_time = "\t\tQuery Execution [ms]"
        data_transfer_time = "\t\tData Transfer [ms]"
        post_join_time = "\t\tPost-join [ms]"
        sum_time = "\t\t$\\sum$ [ms]"
        for q in tabular_queries:
            qet_st = data[data['query'] == q]['st_exec_time'].values[0]
            qet_rm = data[data['query'] == q]['rm_exec_time'].values[0]
            dt_st = data[data['query'] == q]['st_transfer_time'].values[0]
            dt_rm = data[data['query'] == q]['rm_transfer_time'].values[0]
            pj_rm = data[data['query'] == q]['rm_post_join_time'].values[0]
            query_execution_time += f" & {round(qet_st, 2)} & {round(qet_rm, 2)}"
            data_transfer_time += f" & {round(dt_st, 2)} & {round(dt_rm, 2)}"
            post_join_time += f" & - & {round(pj_rm, 2)}"
            sum_time += f" & {round(qet_st + dt_st, 2)} & {round(qet_rm + dt_rm + pj_rm, 2)}"
        tabular += f"{query_execution_time}\\\\\n"
        tabular += f"{data_transfer_time}\\\\\n"
        tabular += f"{post_join_time}\\\\\\hline\n"
        tabular += f"{sum_time}\\\\\\hline\n"
        tabular += f"\t\\end{{tabular}}\n"
    table_bottom = f"\\end{{table*}}\n"
    latex = f"{table_top}{tabular}{table_bottom}"
    print(latex)

queries = job_subset

data = pd.read_csv('../rewrite-methods/job/rewrite-results.csv')
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer', 'run'], as_index=False).sum()
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer'], as_index=False).median().drop(
    ['run', 'num_query_internal'], axis=1)
data = data.drop(['database', 'system'], axis=1)
data  = data[data['query'].isin(queries)]
# Remove measurements with data transfer
data = data[data['data_transfer'] == False]

# Extract Single Table query execution times
single_table_execution_time = data[data['method'] == '0. Single Table'].sort_values(by=['query'], key=natsort_keygen())['time'].array

# Compute best RM
data_rm = pd.read_csv('../rewrite-methods/job/rewrite-results-post-join.csv')
data_rm = data_rm.groupby(by=['database', 'system', 'query', 'method', 'data_transfer', 'run'], as_index=False).sum()
data_rm = data_rm.groupby(by=['database', 'system', 'query', 'method', 'data_transfer'], as_index=False).median().drop(
    ['run', 'num_query_internal'], axis=1)
data_rm = data_rm.drop(['database', 'system'], axis=1)
# Remove measurements with data transfer
data_rm = data_rm[data_rm['data_transfer'] == False]
data_rm = data_rm[data_rm['method'] != '0. Single Table']
minimal_idx = data_rm.groupby(by=['query'])['time'].idxmin()
data_rm = data_rm.loc[minimal_idx]
data_rm = data_rm.sort_values(by=['query'], key=natsort_keygen())
best_rm_execution_time = data_rm['time'].array

# Compute result set size for single table and result DB
result_sizes_df = pd.read_csv('../result-set-sizes/job/result-set-sizes.csv')
result_sizes_df = result_sizes_df[result_sizes_df['query'].isin(queries)]
result_sizes_df = result_sizes_df.drop(['relation', 'count'], axis=1)
result_sizes_df = result_sizes_df.groupby(by=['database', 'query', 'method'], as_index=False).sum()
result_sizes_df = result_sizes_df.sort_values(by=['query', 'method'], key=natsort_keygen())
result_sizes_df['size'] = round(result_sizes_df['size'] / KB, 2)

st_result_sizes = result_sizes_df[result_sizes_df['method'] == 'Single Table']['size'].array
rm_result_sizes = result_sizes_df[result_sizes_df['method'] == 'rdb_w_post_join_info']['size'].array

# Compute the data transfer time for a given data transfer rate
data_transfer_rate = 100 # Mbps
st_transfer_times = [(((size / 1024) * 8) / data_transfer_rate) * 1000 for size in st_result_sizes]
rm_transfer_times = [(((size / 1024) * 8) / data_transfer_rate) * 1000 for size in rm_result_sizes]

# Extract post-join times
post_join_times_df = pd.read_csv('../post-join/job/postjoin-postgres-results.csv')
post_join_times_df = post_join_times_df[post_join_times_df['query'].isin(queries)]
post_join_times_df = post_join_times_df.groupby(by=['database', 'system', 'method', 'query'], as_index=False).median().drop(['database', 'system', 'method', 'run'], axis=1)
post_join_times_df = post_join_times_df.sort_values(by=['query'], key=natsort_keygen())
post_join_times = post_join_times_df['time'].array

# Construct final dataframe 
data = pd.DataFrame(data= {
    'query': data_rm['query'],
    'st_exec_time': single_table_execution_time,
    'rm_exec_time': best_rm_execution_time,
    'st_transfer_time': st_transfer_times,
    'rm_transfer_time': rm_transfer_times,
    'rm_post_join_time': post_join_times,
}).reset_index(drop=True)

to_latex(data, 5)

##  Algorithm

### Join-Order Benchmark

In [ ]:
def bar_plot(data, filename):
    queries = data['query']
    x = np.arange(len(queries))  # the label locations
    width = 0.20  # the width of the bars
    fig, ax = plt.subplots(figsize=set_size(fraction_width=1, fraction_height=0.2), layout='constrained')
    
    # Single Table
    ax.bar(x, data['st_exec_time'], width, color=colormap['Single Table'], edgecolor='black',label='Single Table')
    ax.bar(x, data['st_transfer_time'],  width=width, bottom=data['st_exec_time'], color=colormap['ST transfer time'], edgecolor='black', label='Single Table Transfer Time')
    
    # Rewrite Method
    ax.bar(x + width, data['rdb_exec_time'], width=width, color=colormap['Result DB'], edgecolor='black', label='Result DB')
    ax.bar(x + width, data['rdb_transfer_time'], width=width, bottom=data['rdb_exec_time'], color=colormap['RDB transfer time'], edgecolor='black', label='Result DB Transfer Time')
    ax.bar(x + width, data['rdb_post_join_time'], width=width, bottom=data['rdb_exec_time'] + data['rdb_transfer_time'], color=colormap['RDB post-join'], edgecolor='black', label='Result DB Post-join')
    
    ax.set_xticks(x + width / 2, queries)
    ax.set_xlabel('JOB queries')
    ax.legend(loc='upper right')
    ax.xaxis.grid(False)  # Disable grid lines on the x-axis

    log_scale = False
    if log_scale:
        ax.set_ylabel('Runtime [ms] (log-scale)')
        ax.set_yscale('log')
    else:
        ax.set_ylabel('Runtime [ms]')
        ax.set_ylim(0)

    fig.savefig(filename, bbox_inches='tight')


def to_latex(data, entries_per_row):
    table_top = f"\\begin{{table*}}\n"
    table_top += f"\t\\small\n"
    table_top += f"\t\\caption{{End-to-end performance of the \\textsc{{ResultDB}}$_{{\\textsc{{semi-join}}}}$ algorithm compared to the Single Table (ST) execution on JOB.}} \n"
    table_top += f"\t\\label{{tab:job-end-to-end-result-db}}\n"

    tabular = ""
    queries = natsorted(data['query'].unique())
    num_queries = len(queries)
    assert num_queries % entries_per_row == 0, "number of queries should be a multiple of entries per row"
    num_tabulars = num_queries // entries_per_row
    for t in range(num_tabulars):
        tabular_queries = queries[t * entries_per_row:t * entries_per_row + entries_per_row]
        tabular += f"\t\\begin{{tabular}}{{\n"
        tabular += f"\t\t|l!{{\\vrule width 1.5pt}}\n"
        for e in range(entries_per_row):
            if e != entries_per_row - 1:
                tabular += f"\t\tR{{\\cellwidth}}|R{{\\cellwidth}}!{{\\vrule width \\vw}}\n"
            else:
                tabular += f"\t\tR{{\\cellwidth}}|R{{\\cellwidth}}|}}\n"
        tabular += f"\t\t\\hline\n"
        tabular += f"\t\t\\textbf{{JOB Query}}\n"
        for i, q in enumerate(tabular_queries):
            if i != entries_per_row - 1:
                tabular += f"\t\t& \\multicolumn{{2}}{{c!{{\\vrule width \\vw}}}}{{\\textbf{{{q[1:]}}}}}\n"
            else:
                tabular += f"\t\t& \\multicolumn{{2}}{{c|}}{{\\textbf{{{q[1:]}}}}}\n"
        tabular += f"\t\t\\\\\\hline\n"
        tabular += f"\t\t\\textbf{{Approach}}"
        for e in range(entries_per_row):
            if e != entries_per_row - 1:
                tabular += f"\t\t& \\multicolumn{{1}}{{c|}}{{\\textbf{{ST}}}} & \\multicolumn{{1}}{{c!{{\\vrule width \\vw}}}}{{\\textbf{{RDB$_{{\\text{{RP}}}}$}}}}\n"
            else:
                tabular += f"\t\t& \\multicolumn{{1}}{{c|}}{{\\textbf{{ST}}}} & \\multicolumn{{1}}{{c|}}{{\\textbf{{RDB$_{{\\text{{RP}}}}$}}}}\n"
        tabular += f"\t\t\\\\ \\noalign{{\\hrule height 1.5pt}}\n"
        query_execution_time = "\t\tQuery Execution [ms]"
        data_transfer_time = "\t\tData Transfer [ms]"
        post_join_time = "\t\tPost-join [ms]"
        sum_time = "\t\t$\\sum$ [ms]"
        for q in tabular_queries:
            qet_st = data[data['query'] == q]['st_exec_time'].values[0]
            qet_rdb = data[data['query'] == q]['rdb_exec_time'].values[0]
            dt_st = data[data['query'] == q]['st_transfer_time'].values[0]
            dt_rdb = data[data['query'] == q]['rdb_transfer_time'].values[0]
            pj_rdb = data[data['query'] == q]['rdb_post_join_time'].values[0]
            query_execution_time += f" & {round(qet_st, 2)} & {round(qet_rdb, 2)}"
            data_transfer_time += f" & {round(dt_st, 2)} & {round(dt_rdb, 2)}"
            post_join_time += f" & - & {round(pj_rdb, 2)}"
            sum_time += f" & {round(qet_st + dt_st, 2)} & {round(qet_rdb + dt_rdb + pj_rdb, 2)}"
        tabular += f"{query_execution_time}\\\\\n"
        tabular += f"{data_transfer_time}\\\\\n"
        tabular += f"{post_join_time}\\\\\\hline\n"
        tabular += f"{sum_time}\\\\\\hline\n"
        tabular += f"\t\\end{{tabular}}\n"
    table_bottom = f"\\end{{table*}}\n"
    latex = f"{table_top}{tabular}{table_bottom}"
    print(latex)

# Extract Single Table and Result DB execution times in mutable
# ensure that `queries` is sorted!
queries = [q for q in job_subset if q != "q16b"]

st_exec_times = []
rdb_exec_times = []
processed_queries = []
num_queries = 0
# TODO: update the mutable results! currently, we compute RDB and not RDB_RP, i.e., we do not have a relationship-preserving database
for query in queries:
    result_file = Path(f"../algorithm/job/{query}_results.csv")
    if not result_file.is_file():
        print(f"{result_file} does not exist. Skipping...")
        continue
    processed_queries.append(query)
    data = pd.read_csv(result_file)  
    data = (data.groupby(by=['commit', 'date', 'version', 'suite', 'benchmark', 'experiment', 'name', 'config', 'case',], as_index=False)
                .median()
                .drop(['commit', 'date', 'version', 'suite', 'benchmark', 'case', 'runid'], axis=1)
           )
    experiment = data['experiment'].unique()
    assert len(experiment) == 1, f'experiment contains multiple different queries'
    assert experiment[0] == query, f'experiment query: {experiment[0]} does not match query {query} of current file'

    for _, row in data.iterrows():
        algorithm = row['name']
        execution_time = row['time']
        if "single-table" in algorithm:
            single_table_execution_time = execution_time
        elif "decompose" in algorithm:
            pass
        elif "resultdb" in algorithm:
            resultdb_execution_time = execution_time
        else:
            assert False, "experiment name: {name} does not match any of our algorithms"

    st_exec_times.append(single_table_execution_time)
    rdb_exec_times.append(resultdb_execution_time)

assert len(st_exec_times) == len(rdb_exec_times) == len(processed_queries), f'number of measurements has to match the processed queries'

# Compute result set size for single table and ResultDB
result_sizes_df = pd.read_csv('../result-set-sizes/job/result-set-sizes.csv')
result_sizes_df = result_sizes_df.drop(['relation', 'count'], axis=1)
result_sizes_df = result_sizes_df.groupby(by=['database', 'query', 'method'], as_index=False).sum()
result_sizes_df = result_sizes_df.sort_values(by=['query', 'method'], key=natsort_keygen())
result_sizes_df = result_sizes_df[result_sizes_df['query'].isin(queries)]
result_sizes_df['size'] = round(result_sizes_df['size'] / KB, 2)

st_result_sizes = result_sizes_df[result_sizes_df['method'] == 'Single Table']['size'].array
rdb_result_sizes = result_sizes_df[result_sizes_df['method'] == 'rdb_w_post_join_info']['size'].array

# Compute the data transfer time for a given data transfer rate
data_transfer_rate = 100 # Mbps
st_transfer_times = [(((size / 1024) * 8) / data_transfer_rate) * 1000 for size in st_result_sizes]
rdb_transfer_times = [(((size / 1024) * 8) / data_transfer_rate) * 1000 for size in rdb_result_sizes]

# Extract post-join times
post_join_times = []
processed_queries = []
num_queries = 0
for query in queries:
    result_file = Path(f"../post-join/job/{query}/{query}_results.csv")
    if not result_file.is_file():
        print(f"{result_file} does not exist. Skipping...")
        continue
    processed_queries.append(query)
    data = pd.read_csv(result_file)  
    data = (data.groupby(by=['commit', 'date', 'version', 'suite', 'benchmark', 'experiment', 'name', 'config', 'case',], as_index=False)
                .median()
                .drop(['commit', 'date', 'version', 'suite', 'benchmark', 'case', 'runid'], axis=1)
           )
    experiment = data['experiment'].unique()
    assert len(experiment) == 1, f'experiment contains multiple different queries'
    assert experiment[0] == query, f'experiment query: {experiment[0]} does not match query {query} of current file'

    assert data.shape[0] == 1, f'after aggregating, dataframe should only contain one row'
    
    post_join_times.append(data.iloc[0]['time'])

# Construct final dataframe 
data = pd.DataFrame(data= {
    'query': queries,
    'st_exec_time': st_exec_times,
    'rdb_exec_time': rdb_exec_times,
    'st_transfer_time': st_transfer_times,
    'rdb_transfer_time': rdb_transfer_times,
    'rdb_post_join_time': post_join_times,
}).reset_index(drop=True)

to_latex(data, 3)